# Capital Markets Workflow (Simplified)

In [ ]:
# 02/21/2025 18:05:00

In [307]:
# Install packages
%pip install -U --quiet yfinance pandas 
%pip install -U --quiet langgraph langchain-community langchain-aws langchain-mongodb
%pip install -U --quiet pymongo
%pip install -U --quiet python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Set environment variables

In [308]:
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Set environment variables
MONGODB_URI = os.getenv("MONGODB_URI")
AWS_REGION = os.getenv("AWS_REGION")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

# Set database name
DATABASE_NAME = "agentic_capital_markets"


## Getting Market Data from yfinance

### yfinance documentation
https://ranaroussi.github.io/yfinance/reference/

Yahoo!, Y!Finance, and Yahoo! finance are registered trademarks of Yahoo, Inc.

yfinance is not affiliated, endorsed, or vetted by Yahoo, Inc. It’s an open-source tool that uses Yahoo’s publicly available APIs, and is intended for research and educational purposes.

You should refer to Yahoo!’s terms of use (here), (here), and (here) for details on your rights to use the actual data downloaded. Remember - the Yahoo! finance API is intended for personal use only.


## Important note

### $SPY: possibly delisted; no price data found  (period=2y) (Yahoo error = "1m data not available for startTime=1676894054 and endTime=1739966054. Only 8 days worth of 1m granularity data are allowed to be fetched per request.")

In [309]:
# Import libraries
import pandas as pd
import yfinance as yf

https://stackoverflow.com/questions/76605981/can-not-download-6-months-tsla-with-interval-of-1-minute-in-yfinance

### Getting equities data

In [310]:
EQUITIES = 'SPY QQQ EEM XLE'

equities = yf.Tickers(EQUITIES)

spy = equities.tickers['SPY'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])
qqq = equities.tickers['QQQ'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])
eem = equities.tickers['EEM'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])
xle = equities.tickers['XLE'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])

spy.tail(5)

,Open,High,Low,Close,Volume
Datetime,,,,,
2025-02-21 11:21:00-05:00,607.359985,607.479980,607.299988,607.419983,73094
2025-02-21 11:22:00-05:00,607.440002,607.460022,607.291199,607.340027,58843
2025-02-21 11:23:00-05:00,607.330017,607.395020,607.280029,607.340027,55311
2025-02-21 11:24:00-05:00,607.335022,607.349976,607.280029,607.344971,55628
2025-02-21 11:25:00-05:00,607.320007,607.320007,607.320007,607.320007,0


### Getting Bonds (U.S. Treasuries) data

In [311]:
BONDS = 'TLT LQD HYG'

bonds = yf.Tickers(BONDS)

tlt = bonds.tickers['TLT'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])
lqd = bonds.tickers['LQD'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])
hyg = bonds.tickers['HYG'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])

tlt.tail(5)

,Open,High,Low,Close,Volume
Datetime,,,,,
2025-02-21 11:21:00-05:00,89.355499,89.355499,89.334999,89.339996,63352
2025-02-21 11:22:00-05:00,89.344498,89.355003,89.324997,89.345596,154396
2025-02-21 11:23:00-05:00,89.345001,89.345001,89.309998,89.320000,65254
2025-02-21 11:24:00-05:00,89.320000,89.324997,89.305000,89.305000,276586
2025-02-21 11:25:00-05:00,89.320000,89.320000,89.320000,89.320000,0


### Getting Commodities data (Gold, Oil, etc.)

In [312]:
COMMODITIES = 'GLD USO'

commodities = yf.Tickers(COMMODITIES)

gld = commodities.tickers['GLD'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])
uso = commodities.tickers['USO'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits', 'Capital Gains'])

gld.tail(5)

,Open,High,Low,Close,Volume
Datetime,,,,,
2025-02-21 11:21:00-05:00,270.519989,270.589996,270.519989,270.570007,4363
2025-02-21 11:22:00-05:00,270.578796,270.589996,270.519989,270.519989,9451
2025-02-21 11:23:00-05:00,270.509796,270.619995,270.509796,270.600006,9800
2025-02-21 11:24:00-05:00,270.600006,270.630005,270.572693,270.572693,8669
2025-02-21 11:25:00-05:00,270.559998,270.559998,270.559998,270.559998,0


### Getting Market Volatility Index data

In [313]:
MARKET_VOLATILITY = '^VIX'

market_volatility = yf.Tickers(MARKET_VOLATILITY)

vix = market_volatility.tickers['^VIX'].history(period='max', interval='1m').drop(columns=['Dividends', 'Stock Splits'])

vix.tail(5)

,Open,High,Low,Close,Volume
Datetime,,,,,
2025-02-21 10:06:00-06:00,16.030001,16.049999,16.030001,16.030001,0
2025-02-21 10:07:00-06:00,16.020000,16.020000,15.930000,15.930000,0
2025-02-21 10:08:00-06:00,15.940000,16.020000,15.940000,16.020000,0
2025-02-21 10:09:00-06:00,15.970000,15.990000,15.950000,15.990000,0
2025-02-21 10:10:00-06:00,15.990000,15.990000,15.990000,15.990000,0


## We can leverage Scheduled Triggers in MongoDB Atlas to run the script at a specific recurring time.

Scheduled Triggers allow you to execute server-side logic on a regular schedule that you define. You can use scheduled Triggers to do work that happens on a periodic basis, such as updating a document every minute, generating a nightly report, or sending an automated weekly email newsletter.

https://www.mongodb.com/docs/atlas/atlas-ui/triggers/scheduled-triggers/

## We can leverage Atlas Functions as well

An Atlas Function is a piece of server-side JavaScript code that you write to define your app's behavior. You can call your app's Functions directly from a client app or define services that integrate and call Functions automatically.

Functions can call other Functions and include a built-in client for working with data in MongoDB Atlas clusters. They also include helpful global utilities, support common Node.js built-in modules, and can import and use external packages from the npm registry.

https://www.mongodb.com/docs/atlas/atlas-ui/triggers/functions/

Video Tutorial: https://www.youtube.com/watch?v=3Q6J1J1JZ3A

## MongoDB Database Setup

In [314]:
# Import libraries
from pymongo import MongoClient, ASCENDING

In [315]:
DATABASE_NAME = "agentic_capital_markets"

def get_mongo_client(mongo_uri):
    """Establish connection to the MongoDB and ping the database."""

    # gateway to interacting with a MongoDB database cluster
    client = MongoClient(mongo_uri, appname="ist.demo.agentic_capital_markets.python", tz_aware=True)

    # Ping the database to ensure the connection is successful
    try:
        client.admin.command("ping")
        print("Connection to MongoDB successful")
    except Exception as e:
        print(f"Error connecting to MongoDB: {e}")
        return None

    return client


if not MONGODB_URI:
    print("MONGO_URI not set in environment variables")

mongo_client = get_mongo_client(MONGODB_URI)

if mongo_client:
    # Pymongo client of database
    db = mongo_client.get_database(DATABASE_NAME)
else:
    print("Failed to connect to MongoDB. Exiting...")
    exit(1)

Connection to MongoDB successful


### Create Timeseries Collection

In [ ]:
# Check if the 'yfinanceMarketData' collection exists
if "yfinanceMarketData" in db.list_collection_names():
    db.yfinanceMarketData.drop()  # ⚠️ This will delete all data!

from bson.codec_options import CodecOptions
from bson.datetime_ms import DatetimeConversion

# For more information on datetime conversion options, see:
# https://www.mongodb.com/docs/languages/python/pymongo-driver/current/data-formats/dates-and-times/
codec_options = CodecOptions(datetime_conversion=DatetimeConversion.DATETIME_AUTO)

# Check if the 'yfinanceMarketData' collection exists
if "yfinanceMarketData" not in db.list_collection_names():
    # Create the collection with a time-series configuration
    db.create_collection(
        "yfinanceMarketData",
        timeseries={
            "timeField": "timestamp",
            "metaField": "symbol",
            "granularity": "minutes"
        },
        codec_options=codec_options
        # expireAfterSeconds=604800  # Document expires 7 days (604800 seconds)
    )
    
    # Create an index on the `symbol` and `timestamp` fields
    db.yfinanceMarketData.create_index(
        [("symbol", ASCENDING), ("timestamp", ASCENDING)]
    )
    
    print("Collection and index created successfully.")
else:
    print("The 'yfinanceMarketData' collection already exists.")

Collection and index created successfully.


## Data Ingestion

In [319]:
# Checking pandas DataFrame info
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1557 entries, 2025-02-14 11:26:00-05:00 to 2025-02-21 11:25:00-05:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1557 non-null   float64
 1   High    1557 non-null   float64
 2   Low     1557 non-null   float64
 3   Close   1557 non-null   float64
 4   Volume  1557 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 73.0 KB


###  Transformations

In [320]:
from datetime import datetime
import pytz


def transform_data(symbol, df):
    """
    Transforms the given DataFrame by performing the following operations:
        - Reset the index (Datetime) and rename it to 'timestamp'.
        - Add a new column 'symbol' with the given value.
        - Convert all column names to lowercase.
        - Reorder columns as 'timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume'.
        - Convert 'timestamp' to UTC timezone if not already.
    Parameters:
        - symbol (str): The symbol name to be added in the 'symbol' column.
        - df (pd.DataFrame): The input DataFrame with 'Datetime' as the index.
    Returns:
        - pd.DataFrame: Transformed DataFrame.
    """
    # Reset index and explicitly rename 'Datetime' to 'timestamp'
    df = df.reset_index().rename(columns={'Datetime': 'timestamp'})

    # Convert timestamp to datetime if not already
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Ensure timestamp is localized to US/Eastern only if it’s naive (has no timezone)
    if df['timestamp'].dt.tz is None:
        eastern = pytz.timezone('US/Eastern')
        df['timestamp'] = df['timestamp'].dt.tz_localize(
            eastern, ambiguous='NaT', nonexistent='shift_forward')

    # Convert 'timestamp' to UTC
    df['timestamp'] = df['timestamp'].dt.tz_convert('UTC')

    # Add the 'symbol' column
    df['symbol'] = symbol

    # Convert all column names to lowercase
    df.columns = df.columns.str.lower()

    # Reorder columns
    df = df[['timestamp', 'symbol', 'open', 'high', 'low', 'close', 'volume']]

    return df

In [321]:
# Equity transformation - SPY as example
df_spy = transform_data('SPY-USD', spy)
df_spy.head()

,timestamp,symbol,open,high,low,close,volume
0,2025-02-14 16:26:00+00:00,SPY-USD,609.979980,610.059998,609.919983,609.989990,0
1,2025-02-14 16:27:00+00:00,SPY-USD,609.984985,610.250000,609.984985,610.169983,45942
2,2025-02-14 16:28:00+00:00,SPY-USD,610.200012,610.221985,610.030029,610.030029,32481
3,2025-02-14 16:29:00+00:00,SPY-USD,610.080017,610.130005,609.989990,609.989990,22067
4,2025-02-14 16:30:00+00:00,SPY-USD,610.010010,610.020020,609.702820,609.789978,51777


In [322]:
df_spy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1557 entries, 0 to 1556
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   timestamp  1557 non-null   datetime64[ns, UTC]
 1   symbol     1557 non-null   object             
 2   open       1557 non-null   float64            
 3   high       1557 non-null   float64            
 4   low        1557 non-null   float64            
 5   close      1557 non-null   float64            
 6   volume     1557 non-null   int64              
dtypes: datetime64[ns, UTC](1), float64(4), int64(1), object(1)
memory usage: 85.3+ KB


In [323]:
# Equity transformations
df_qqq = transform_data('QQQ-USD', qqq)
df_eem = transform_data('EEM-USD', eem)
df_xle = transform_data('XLE-USD', xle)

# Bonds transformations
df_tlt = transform_data('TLT-USD', tlt)
df_lqd = transform_data('LQD-USD', lqd)
df_hyg = transform_data('HYG-USD', hyg)

# Commodities transformations
df_gld = transform_data('GLD-USD', gld)
df_uso = transform_data('USO-USD', uso)

# Market Volatility transformations
df_vix = transform_data('VIX-USD', vix)

In [324]:
df_uso.sort_index(ascending=False).head()

,timestamp,symbol,open,high,low,close,volume
1517,2025-02-21 16:25:00+00:00,USO-USD,76.059998,76.059998,76.059998,76.059998,0
1516,2025-02-21 16:24:00+00:00,USO-USD,76.059998,76.065002,76.059998,76.065002,2209
1515,2025-02-21 16:23:00+00:00,USO-USD,76.050003,76.050003,76.044998,76.044998,2276
1514,2025-02-21 16:22:00+00:00,USO-USD,76.059998,76.059998,76.059998,76.059998,913
1513,2025-02-21 16:21:00+00:00,USO-USD,76.084999,76.084999,76.084999,76.084999,1699


### Clean Data

In [325]:
# Ensure that the collection is empty by deleting all documents
db.yfinanceMarketData.delete_many({})

print("The 'yfinanceMarketData' collection is now empty.")

The 'yfinanceMarketData' collection is now empty.


### Insert data into MongoDB time series collection

In [326]:
# df_spy sort by timestamp in ascending order
df_spy = df_spy.sort_index(ascending=True)
df_spy.tail(20)

,timestamp,symbol,open,high,low,close,volume
1537,2025-02-21 16:06:00+00:00,SPY-USD,606.559998,606.609985,606.440308,606.530029,37127
1538,2025-02-21 16:07:00+00:00,SPY-USD,606.505005,606.853271,606.469971,606.760010,49458
1539,2025-02-21 16:08:00+00:00,SPY-USD,606.770020,606.799988,606.370911,606.599976,312843
1540,2025-02-21 16:09:00+00:00,SPY-USD,606.590027,606.750000,606.520081,606.565002,2927471
1541,2025-02-21 16:10:00+00:00,SPY-USD,606.530029,606.530029,606.179993,606.310974,0
1542,2025-02-21 16:11:00+00:00,SPY-USD,606.390015,606.487488,606.285095,606.460022,41464
1543,2025-02-21 16:12:00+00:00,SPY-USD,606.500000,606.609985,606.400024,606.609985,69644
1544,2025-02-21 16:13:00+00:00,SPY-USD,606.625000,606.880005,606.570007,606.809998,62842
1545,2025-02-21 16:14:00+00:00,SPY-USD,606.869995,607.140015,606.869995,607.140015,76004
1546,2025-02-21 16:15:00+00:00,SPY-USD,607.159973,607.250000,607.054993,607.080017,86697


In [327]:
# df_spy sort by timestamp in descending order
df_spy = df_spy.sort_index(ascending=False)
df_spy.tail(20)

,timestamp,symbol,open,high,low,close,volume
19,2025-02-14 16:45:00+00:00,SPY-USD,610.065002,610.080017,610.030090,610.065002,14474
18,2025-02-14 16:44:00+00:00,SPY-USD,610.020020,610.085022,609.979980,610.080017,33125
17,2025-02-14 16:43:00+00:00,SPY-USD,610.039978,610.119995,610.030029,610.030029,33947
16,2025-02-14 16:42:00+00:00,SPY-USD,610.049988,610.065002,610.020020,610.030029,15087
15,2025-02-14 16:41:00+00:00,SPY-USD,610.010010,610.072388,609.950012,610.039978,149082
14,2025-02-14 16:40:00+00:00,SPY-USD,610.020020,610.020020,610.020020,610.020020,24279
13,2025-02-14 16:39:00+00:00,SPY-USD,609.909973,609.979980,609.885010,609.965271,19598
12,2025-02-14 16:38:00+00:00,SPY-USD,609.815002,609.929993,609.794983,609.900024,37241
11,2025-02-14 16:37:00+00:00,SPY-USD,609.669983,609.820007,609.659973,609.815002,39518
10,2025-02-14 16:36:00+00:00,SPY-USD,609.969971,609.979980,609.630005,609.710022,39771


In [ ]:
# df_spy.to_csv('spy_7d_1m_data.csv', index=False)

In [328]:
# Pandas DataFrame to dictionary, example with SPY
spy_records = df_spy.to_dict(orient="records")
spy_records

[{'timestamp': Timestamp('2025-02-21 16:25:00+0000', tz='UTC'),
  'symbol': 'SPY-USD',
  'open': 607.3200073242188,
  'high': 607.3200073242188,
  'low': 607.3200073242188,
  'close': 607.3200073242188,
  'volume': 0},
 {'timestamp': Timestamp('2025-02-21 16:24:00+0000', tz='UTC'),
  'symbol': 'SPY-USD',
  'open': 607.3350219726562,
  'high': 607.3499755859375,
  'low': 607.280029296875,
  'close': 607.344970703125,
  'volume': 55628},
 {'timestamp': Timestamp('2025-02-21 16:23:00+0000', tz='UTC'),
  'symbol': 'SPY-USD',
  'open': 607.3300170898438,
  'high': 607.39501953125,
  'low': 607.280029296875,
  'close': 607.3400268554688,
  'volume': 55311},
 {'timestamp': Timestamp('2025-02-21 16:22:00+0000', tz='UTC'),
  'symbol': 'SPY-USD',
  'open': 607.4400024414062,
  'high': 607.4600219726562,
  'low': 607.2911987304688,
  'close': 607.3400268554688,
  'volume': 58843},
 {'timestamp': Timestamp('2025-02-21 16:21:00+0000', tz='UTC'),
  'symbol': 'SPY-USD',
  'open': 607.3599853515625,
 

In [329]:
def insert_market_data(db, df):
    """
    Inserts the transformed DataFrame into the MongoDB time-series collection `yfinanceMarketData`.
    
    Parameters:
        - db (pymongo.database.Database): The MongoDB database object.
        - df (pd.DataFrame): The transformed DataFrame containing market data.
    
    Returns:
    - dict: MongoDB insert result.
    """
    # Confirm timestamp is aware and in UTC
    if df['timestamp'].dt.tz is None:
        # If timestamp is naive, localize to UTC
        df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize('UTC')
    else:
        # If timestamp is aware, convert to UTC
        df['timestamp'] = df['timestamp'].dt.tz_convert('UTC')
    
    # Convert timestamp to Python datetime object (ensuring stored as ISODate)
    df['timestamp'] = df['timestamp'].apply(lambda x: x.to_pydatetime())
    
    # Convert DataFrame to list of dictionaries
    records = df.to_dict(orient="records")
    
    # Insert data into the collection
    result = db.yfinanceMarketData.insert_many(records)
    
    return {"inserted_count": len(result.inserted_ids)}

In [330]:
print("Inserting SPY market data...")
inserted_count = insert_market_data(db=db, df=df_spy)
print(f"Inserted {inserted_count} documents.")

print("Inserting QQQ market data...")
inserted_count = insert_market_data(db=db, df=df_qqq)
print(f"Inserted {inserted_count} documents.")

print("Inserting EEM market data...")
inserted_count = insert_market_data(db=db, df=df_eem)
print(f"Inserted {inserted_count} documents.")

print("Inserting XLE market data...")
inserted_count = insert_market_data(db=db, df=df_xle)
print(f"Inserted {inserted_count} documents.")

print("Inserting TLT market data...")
inserted_count = insert_market_data(db=db, df=df_tlt)
print(f"Inserted {inserted_count} documents.")

print("Inserting LQD market data...")
inserted_count = insert_market_data(db=db, df=df_lqd)
print(f"Inserted {inserted_count} documents.")

print("Inserting HYG market data...")
inserted_count = insert_market_data(db=db, df=df_hyg)
print(f"Inserted {inserted_count} documents.")

print("Inserting GLD market data...")
inserted_count = insert_market_data(db=db, df=df_gld)
print(f"Inserted {inserted_count} documents.")

print("Inserting USO market data...")
inserted_count = insert_market_data(db=db, df=df_uso)
print(f"Inserted {inserted_count} documents.")

print("Inserting VIX market data...")
inserted_count = insert_market_data(db=db, df=df_vix)
print(f"Inserted {inserted_count} documents.")

Inserting SPY market data...
Inserted {'inserted_count': 1557} documents.
Inserting QQQ market data...
Inserted {'inserted_count': 1557} documents.
Inserting EEM market data...
Inserted {'inserted_count': 1556} documents.
Inserting XLE market data...
Inserted {'inserted_count': 1557} documents.
Inserting TLT market data...
Inserted {'inserted_count': 1556} documents.
Inserting LQD market data...
Inserted {'inserted_count': 1553} documents.
Inserting HYG market data...
Inserted {'inserted_count': 1556} documents.
Inserting GLD market data...
Inserted {'inserted_count': 1556} documents.
Inserting USO market data...
Inserted {'inserted_count': 1518} documents.
Inserting VIX market data...
Inserted {'inserted_count': 3560} documents.


In [331]:
from pprint import pprint

# Query and sort data by timestamp in descending order
cursor = db.yfinanceMarketData.find({"symbol": "SPY-USD"}).sort("timestamp", -1)

# Pretty-print the results
for document in cursor:
    pprint(document)

{'_id': ObjectId('67b8a90081f9b9d84329302c'),
 'close': 607.3200073242188,
 'high': 607.3200073242188,
 'low': 607.3200073242188,
 'open': 607.3200073242188,
 'symbol': 'SPY-USD',
 'timestamp': datetime.datetime(2025, 2, 21, 16, 25, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC')),
 'volume': 0}
{'_id': ObjectId('67b8a90081f9b9d84329302d'),
 'close': 607.344970703125,
 'high': 607.3499755859375,
 'low': 607.280029296875,
 'open': 607.3350219726562,
 'symbol': 'SPY-USD',
 'timestamp': datetime.datetime(2025, 2, 21, 16, 24, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC')),
 'volume': 55628}
{'_id': ObjectId('67b8a90081f9b9d84329302e'),
 'close': 607.3400268554688,
 'high': 607.39501953125,
 'low': 607.280029296875,
 'open': 607.3300170898438,
 'symbol': 'SPY-USD',
 'timestamp': datetime.datetime(2025, 2, 21, 16, 23, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC')),
 'volume': 55311}
{'_id': ObjectId('67b8a90081f9b9d84329302f'),
 'close': 607.3400268554688,
 'high': 607.4600219726562,


### Data Aggregation

In [332]:
# Aggregate the data by hour
# Hourly aggregation to get high, low, open, and close values
hourly_aggregation = db.yfinanceMarketData.aggregate([
    {
        "$match": { "symbol": "SPY-USD" }
    },
    {
        "$group": {
            "_id": {
                "symbol": "$symbol",
                "time": {
                    "$dateTrunc": {
                        "date": "$timestamp",
                        "unit": "hour",
                        "binSize": 1
                    }
                }
            },
            "high": { "$max": "$high" },
            "low": { "$min": "$low" },
            "open": { "$first": "$open" },
            "close": { "$last": "$close" }
        }
    },
    {
        "$sort": { "_id.time": 1 }
    }
])

# Store the aggregation result in a list
hourly_results = list(hourly_aggregation)

# Pretty-print the results
for document in hourly_results:
    pprint(document)

# Count the number of documents in the results
print(f"Total documents: {len(hourly_results)}")


{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 16, 0, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 610.22998046875,
 'high': 610.2949829101562,
 'low': 609.6099853515625,
 'open': 609.97998046875}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 17, 0, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 609.9199829101562,
 'high': 610.2449951171875,
 'low': 609.0700073242188,
 'open': 610.219970703125}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 18, 0, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 610.47998046875,
 'high': 610.7506713867188,
 'low': 609.7000122070312,
 'open': 609.9199829101562}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 19, 0, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 610.219970703125,
 'high': 610.719970703125,
 'low': 609.8300170898438,
 'open': 610.4899291992188}
{'_id': {'sym

### Calculating Moving Average (MA) - Granulatity unit: hour / binSize: 1 (1 hour)

In [338]:
# Second step: Calculate Moving Average (MA3) from the hourly aggregated data
aggr_with_ma = db.yfinanceMarketData.aggregate([
    {
        "$match": { "symbol": "SPY-USD" }
    },
    {
        "$group": {
            "_id": {
                "symbol": "$symbol",
                "time": {
                    "$dateTrunc": {
                        "date": "$timestamp",
                        "unit": "hour",
                        "binSize": 1
                    }
                }
            },
            "high": { "$max": "$high" },
            "low": { "$min": "$low" },
            "open": { "$first": "$open" },
            "close": { "$last": "$close" }
        }
    },
    {
        "$sort": { "_id.time": 1 }
    },
    {
        "$setWindowFields": {
            "partitionBy": "$_id.symbol",  # Partition by symbol
            "sortBy": { "_id.time": 1 },  # Sort by time (ascending)
            "output": {
                "ma3": {
                    "$avg": "$close",  # Calculate the average of 'close' field
                    "window": { "documents": [-2, 0] }  # Look at the previous 2 documents and current document
                }
            }
        }
    }
])

# Store the aggregation result in a list
aggr_with_ma_results = list(aggr_with_ma)

# Pretty-print the results with moving average
for document in aggr_with_ma_results:
    pprint(document)

# Count the number of documents in the results
print(f"Total documents: {len(aggr_with_ma_results)}")

{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 16, 0, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 610.22998046875,
 'high': 610.2949829101562,
 'low': 609.6099853515625,
 'ma3': 610.22998046875,
 'open': 609.97998046875}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 17, 0, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 609.9199829101562,
 'high': 610.2449951171875,
 'low': 609.0700073242188,
 'ma3': 610.0749816894531,
 'open': 610.219970703125}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 18, 0, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 610.47998046875,
 'high': 610.7506713867188,
 'low': 609.7000122070312,
 'ma3': 610.2099812825521,
 'open': 609.9199829101562}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 19, 0, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 610.219970703125,
 'high': 610.719

### Calculating Moving Average (MA) - Granulatity unit: minute / binSize: 5 (5 minutes)

In [339]:
# Second step: Calculate Moving Average (MA3) from the hourly aggregated data
aggr_with_ma = db.yfinanceMarketData.aggregate([
    {
        "$match": { "symbol": "SPY-USD" }
    },
    {
        "$group": {
            "_id": {
                "symbol": "$symbol",
                "time": {
                    "$dateTrunc": {
                        "date": "$timestamp",
                        "unit": "minute",
                        "binSize": 5
                    }
                }
            },
            "high": { "$max": "$high" },
            "low": { "$min": "$low" },
            "open": { "$first": "$open" },
            "close": { "$last": "$close" }
        }
    },
    {
        "$sort": { "_id.time": 1 }
    },
    {
        "$setWindowFields": {
            "partitionBy": "$_id.symbol",  # Partition by symbol
            "sortBy": { "_id.time": 1 },  # Sort by time (ascending)
            "output": {
                "ma3": {
                    "$avg": "$close",  # Calculate the average of 'close' field
                    "window": { "documents": [-2, 0] }  # Look at the previous 2 documents and current document
                }
            }
        }
    }
])

# Store the aggregation result in a list
aggr_with_ma_results = list(aggr_with_ma)

# Pretty-print the results with moving average
for document in aggr_with_ma_results:
    pprint(document)

# Count the number of documents in the results
print(f"Total documents: {len(aggr_with_ma_results)}")

{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 16, 25, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 609.989990234375,
 'high': 610.25,
 'low': 609.9199829101562,
 'ma3': 609.989990234375,
 'open': 609.97998046875}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 16, 30, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 609.8900146484375,
 'high': 610.02001953125,
 'low': 609.6099853515625,
 'ma3': 609.9400024414062,
 'open': 610.010009765625}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 16, 35, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 609.9652709960938,
 'high': 609.97998046875,
 'low': 609.6300048828125,
 'ma3': 609.9484252929688,
 'open': 609.8800048828125}
{'_id': {'symbol': 'SPY-USD',
         'time': datetime.datetime(2025, 2, 14, 16, 40, tzinfo=FixedOffset(datetime.timedelta(0), 'UTC'))},
 'close': 610.0800170898438,
 'high': 610.119995117

### Financial Analyst note

I'm not a financial expert, but I think that financial analyts use different parametersfor the Moving Average calculation

mostly I saw that they use 200 days of moving average and if the current price is a bit higher than 200 days moving average, then it could be potentially a sell signal

because it's really high, maybe it's overbought, maybe it's time to sell.

### Checking hourly results

In [ ]:
# Pretty-print the results
for document in hourly_results:
    pprint(document)

# Count the number of documents in the results
print(f"Total documents: {len(hourly_results)}")

### Create Time Series Collection for Candlestick Chart

In [ ]:
# Check if the 'yfinanceCandlestickData' collection exists
if "yfinanceCandlestickData" in db.list_collection_names():
    db.yfinanceCandlestickData.drop()  # ⚠️ This will delete all data!

from bson.codec_options import CodecOptions
from bson.datetime_ms import DatetimeConversion

# For more information on datetime conversion options, see:
# https://www.mongodb.com/docs/languages/python/pymongo-driver/current/data-formats/dates-and-times/
codec_options = CodecOptions(datetime_conversion=DatetimeConversion.DATETIME_AUTO)

# Check if the 'yfinanceCandlestickData' collection exists
if "yfinanceCandlestickData" not in db.list_collection_names():
    # Create the collection with a time-series configuration
    db.create_collection(
        "yfinanceCandlestickData",
        timeseries={
            "timeField": "timestamp",
            "metaField": "symbol",
            "granularity": "hours"
        },
        codec_options=codec_options
        # expireAfterSeconds=604800  # Document expires 7 days (604800 seconds)
    )
    
    # Create an index on the `symbol` and `timestamp` fields
    db.yfinanceCandlestickData.create_index(
        [("symbol", ASCENDING), ("timestamp", ASCENDING)]
    )
    
    print("Collection and index created successfully.")
else:
    print("The 'yfinanceCandlestickData' collection already exists.")

In [ ]:
CANDLESTICK_COLLECTION_NAME = "yfinanceCandlestickData"

# Function to store aggregated data
def store_candlestick_data(db, data):
    """
    Store the aggregated candlestick data into a MongoDB collection.
    Parameters:
        - db (pymongo.database.Database): The MongoDB database object.
        - data (list): List of aggregated candlestick data.
    Returns:
        - dict: MongoDB insert result.
    """
    collection = db[CANDLESTICK_COLLECTION_NAME]
    formatted_data = [{
        "symbol": record['_id']['symbol'],
        "timestamp": record['_id']['time'],
        "open": record['open'],
        "high": record['high'],
        "low": record['low'],
        "close": record['close']
    } for record in data]
    result = collection.insert_many(formatted_data)
    return {"inserted_count": len(result.inserted_ids)}

In [345]:
inserted_count = store_candlestick_data(db=db, data=hourly_results)
print(f"Ingesting Candlestick Data...")
print(f"Inserted {inserted_count} documents.")

Ingesting Candlestick Data...
Inserted {'inserted_count': 29} documents.
